# Google Dataset Train npy Generation

In [90]:
!pip install ffmpeg

In [91]:
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
import random
import sys
import io
import os
import glob
import IPython
import wave
import pylab
import pandas as pd
from tf_utils import *
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

# Import files for trigger-word detection model
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

In [92]:
POSITIVE_DIRECTORY = "./raw_data/positive_data/"
BACKGROUND_DIRECTORY = "./raw_data/background_data/"
NEGATIVES_DIRECTORY = "./raw_data/google_dataset/"
NEGATIVES_TRUNCATED_DIRECTORY = "./raw_data/google_dataset_truncated/"
AUDIO_EXAMPLES_DIRECTORY = "./audio_examples/"
AUDIO_IGNORED_EXAMPLES_DIRECTORY = "./audio_ignored_examples/"
POSITIVE_EXAMPLE = "jh_1.wav"
AUDIO_EXAMPLE = "example_train.wav"
STUB_TRAIN_DIRECTORY = "./stub_data/XY_Train/"
STUB_DEV_DIRECTORY = "./stub_data/XY_Dev/"
STUB_MODEL = "./stub_data/models/tr_model.h5"
CONT_EXAMPLE_DIRECTORY = "./cont_example/"
DEV_DIRECTORY = "./raw_data/dev_dataset/"
DEV_CUT_DIRECTORY = "./raw_data/dev_dataset_cut/"
DEV_NPY_DIRECTORY = "./raw_data/dev_npy/"
DEV_GOOGLE_BASED_DATASET_CUT = "./raw_data/dev_google_based_dataset_cut/"
GOOGLE_DATASET_BASED_NPY = "./raw_data/google_dataset_based_npy/"

In [93]:
play(AudioSegment.from_file(DEV_GOOGLE_BASED_DATASET_CUT + "jh_1.wav"))

ValueError: Weird sample rates are not supported.

In [ ]:
Tx = 5490 # Based on created training example
n_freq = 129 # Based on created training example
Ty = 1369 # Based on model.summary() in 1.4 with shape := (Tx, n_freq)

## Some Original Functions

In [ ]:
# Original insert_ones(y,segment_end_ms)
# def insert_ones(y, segment_end_ms):
#     """
#     Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
#     should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
#     50 followinf labels should be ones.
    
    
#     Arguments:
#     y -- numpy array of shape (1, Ty), the labels of the training example
#     segment_end_ms -- the end time of the segment in ms
    
#     Returns:
#     y -- updated labels
#     """
    
#     # duration of the background (in terms of spectrogram time-steps)
#     segment_end_y = int(segment_end_ms * Ty / 10000.0)
#     print(segment_end_y)
    
#     # Add 1 to the correct index in the background label (y)
#     ### START CODE HERE ### (≈ 3 lines)
#     for i in range(segment_end_y + 1, segment_end_y + 51):
#         if i < Ty:
#             y[0, i] = 1
#     ### END CODE HERE ###
    
#     return y

In [ ]:
# Original create_training_example(id)
# def create_training_example(id):
#     """
#     Creates a training examples with a given background, positives, and negatives with id.
    
#     Arguments:
#     id -- an id is given such that the new files does not replace the previous files
    
#     Returns:
#     x -- the spectrogram of the training example
#     y -- the label at each time step of the spectrogram
#     """
#     global BACKGROUND_AUDIONAMES
#     global POSITIVES_AUDIONAMES
#     global NEGATIVES_AUDIONAMES
    
#     background = random_background(BACKGROUND_AUDIONAMES)
#     positives = random_positives(POSITIVES_AUDIONAMES)
#     negatives = random_negatives(NEGATIVES_AUDIONAMES)
    
#     # Make background quieter
#     background = background - 20
    
#     # Step 1: Initialize y (label vector) of zeros (≈ 1 line)
#     y = np.zeros((1, Ty))

#     # Step 2: Initialize segment times as empty list (≈ 1 line)
#     previous_segments = []    
    
#     # Step 3: Loop over randomly selected "activate" clips and insert in background
#     for random_positive in positives:
#         if cannot_insert(random_positive, previous_segments):
#             return create_training_example(id)
#         # Insert the audio clip on the background
#         background, segment_time = insert_audio_clip(background, random_positive, previous_segments)
#         # Retrieve segment_start and segment_end from segment_time
#         segment_start, segment_end = segment_time
#         # Insert labels in "y"
#         y = insert_ones(y, segment_end_ms=segment_end)

#     # Step 4: Loop over randomly selected negative clips and insert in background
#     for random_negative in negatives:
#         if cannot_insert(random_negative, previous_segments):
#             return create_training_example(id)
#         # Insert the audio clip on the background 
#         background, _ = insert_audio_clip(background, random_negative, previous_segments)

#     # Export new training example 
#     background = background.set_channels(1)
#     background = background.set_frame_rate(123000)
    
#     TRAIN_PREFIX = "train_"
#     file_handle = background.export(AUDIO_EXAMPLES_DIRECTORY + TRAIN_PREFIX + str(id) + ".wav", format="wav")
#     # print("File (train_" + str(id) + ".wav) was saved in your directory.")

#     sample_rate, samples = wavfile.read(AUDIO_EXAMPLES_DIRECTORY + TRAIN_PREFIX + str(id) +".wav")
#     frequencies, times, x = signal.spectrogram(samples, sample_rate)
    
#     return frequencies, times, x, y

In [ ]:
# Original create_X_Y(size,start)
# def create_X_Y(size, start):
#     i, X, Y = 1, [], []
#     for i in range(start, start + size):
#         frequencies, times, x, y = create_training_example(i)
#         x = np.transpose(x)
#         y = np.transpose(y)
#         X.append(x)
#         Y.append(y)
#     return (np.array(X), np.array(Y))

## Helper Functions

In [ ]:
def insert_ones_for_human_train_eg(y, segment_ends):
    """
    Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
    should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
    50 followinf labels should be ones.
    
    
    Arguments:
    y -- numpy array of shape (1, Ty), the labels of the training example
    segment_end_ms -- A list of end times segments of the dev file in ms
    
    Returns:
    y -- updated labels
    """

    # duration of the background (in terms of spectrogram time-steps)
    for segment_end_ms in segment_ends:
        segment_end_y = int(segment_end_ms * Ty / 10000.0)

        # Add 1 to the correct index in the background label (y)
        ### START CODE HERE ### (≈ 3 lines)
        for i in range(segment_end_y + 1, segment_end_y + 51):
             if i < Ty:
                 y[0, i] = 1
        ### END CODE HERE ###

    return y

In [ ]:
def sec_to_ms(sec):
    return sec * 1000

In [ ]:
def create_human_train_eg(name,id):
    """
    Creates dev_examples
    Arguments:
    name -- name prefixed to id
    id -- an id is given such that the new files does not replace the previous files
    
    Returns:
    x -- the spectrogram of the training example
    y -- the label at each time step of the spectrogram
    """
    global DEV_GOOGLE_BASED_DATASET_CUT
    
    # Step 1: Initialize y (label vector) of zeros (≈ 1 line)
    y = np.zeros((1, Ty))
    
    # Step 2: Import end time segments    
    segment_ends = pd.read_csv(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) + ".txt", header=None) # Read in the end_time_segments txt
    if not str(segment_ends[0].iloc[0]) == 'nan': # If the audio clip has at least one "basically". Note: "NaN" are given as entries in empty txt
#         print(name + "_" + str(id))
#         print(segment_ends[0])
        segment_ends[0].apply(np.float)
        segment_ends[1] = segment_ends[0].apply(sec_to_ms) # Create a new column of ms
        segment_ends = segment_ends[1].tolist() # List of end time segments 
        
        # Step 3: Insert ones
        #for segment_end in segment_ends:
        y = insert_ones_for_human_train_eg(y, segment_ends)
    
    audio = AudioSegment.from_wav(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) +".wav")
    audio = audio.set_frame_rate(123000)
    file_handle = audio.export(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) +".wav", format = "wav")

    sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) +".wav")
    frequencies, times, x = signal.spectrogram(samples, sample_rate)
    
    return frequencies, times, x, y

In [ ]:
# Test case:
# frequencies, times, x, y = create_dev_example(44)
# c = 0
# for i in y[0]:
#     #print(i)
#     if (i == 1):
#        #print(i)
#        c += 1
# print(c)
#IPython.display.Audio(DEV_CUT_DIRECTORY + "cont_44.wav")

In [103]:
def create_human_train_X_Y(names,list_of_ids):
    """
    names -- list of names
    list_of_ids -- The list of ids of the desired dev audio files
    
    Returns:
    np.array(X)
    np.array(Y)
    """
    X, Y = [], []
    for name in names:
        for id in list_of_ids:
            frequencies, times, x, y = create_human_train_eg(name,id)
            x = np.transpose(x)
            y = np.transpose(y)
            print(name + "_" + str(id))
#             print(x)
#             print("shape of x: " + "x:{}".format(np.array(x).shape))
#             print(y)
            X.append(x)
            Y.append(y)
            print("shape of X: " + "X:{}".format(np.array(X).shape))
    X = np.array(X)
    Y = np.array(Y)
    return (X, Y)

## Create Numpy Arrays for Dev(Continuous) Set

In [100]:
def export_human_train_npy(names,list_of_ids):
    """
    Arguments:
    names -- list of names
    list_of_ids -- The list of ids of the desired dev audio files
    
    Function to save the numpy arrays of the dev audios to DEV_NPY_DIRECTORY
    """
    global GOOGLE_DATASET_BASED_NPY
    
    X, Y = create_human_train_X_Y(names,list_of_ids)
    np.save(GOOGLE_DATASET_BASED_NPY + "human_train_X.npy", X)
    np.save(GOOGLE_DATASET_BASED_NPY + "human_train_Y.npy", Y)
    print("Dimensions of np.array: " + "X:{} Y:{}".format(X.shape, Y.shape))
    print("np.array of X and Y + saved in " + GOOGLE_DATASET_BASED_NPY)

In [101]:
# A list of all currently available ids of dev audios
names = ["Chai", "jh", "larry", "yb"]
list_of_human_train_ids = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [104]:
export_human_train_npy(names, list_of_human_train_ids)

Chai_1
shape of X: X:(1, 5490, 129)
Chai_2
shape of X: X:(2, 5490, 129)
Chai_3
shape of X: X:(3, 5490, 129)
Chai_4
shape of X: X:(4, 5490, 129)
Chai_5
shape of X: X:(5, 5490, 129)
Chai_6
shape of X: X:(6, 5490, 129)
Chai_7
shape of X: X:(7, 5490, 129)
Chai_8
shape of X: X:(8, 5490, 129)
Chai_9
shape of X: X:(9, 5490, 129)
Chai_10
shape of X: X:(10, 5490, 129)
Chai_11
shape of X: X:(11, 5490, 129)
Chai_12
shape of X: X:(12, 5490, 129)
Chai_13
shape of X: X:(13, 5490, 129)
Chai_14
shape of X: X:(14, 5490, 129)
Chai_15
shape of X: X:(15, 5490, 129)
jh_1
shape of X: X:(16, 5490, 129)
jh_2
shape of X: X:(17, 5490, 129)
jh_3
shape of X: X:(18, 5490, 129)
jh_4
shape of X: X:(19, 5490, 129)
jh_5
shape of X: X:(20, 5490, 129)
jh_6
shape of X: X:(21, 5490, 129)
jh_7
shape of X: X:(22, 5490, 129)
jh_8
shape of X: X:(23, 5490, 129)
jh_9
shape of X: X:(24, 5490, 129)
jh_10
shape of X: X:(25, 5490, 129)
jh_11
shape of X: X:(26, 5490, 129)
jh_12
shape of X: X:(27, 5490, 129)
jh_13
shape of X: X:(28, 

In [97]:
# Check Exported npy
print(np.load(GOOGLE_DATASET_BASED_NPY + "human_train_X.npy", allow_pickle=True))
# print(np.load(GOOGLE_DATASET_BASED_NPY + "human_train_Y.npy", allow_pickle=True))

[array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [9.2215399e+02, 2.6837717e+03, 4.0550027e+04, ..., 1.6212604e-03,
        5.9073046e-04, 3.8827796e-04],
       [4.0839306e+01, 1.0362267e+03, 3.3869182e+03, ..., 2.0458781e-04,
        3.1823057e-04, 1.0789970e-04],
       [7.6822609e-02, 9.8626727e+02, 6.9824591e+02, ..., 3.3251880e-04,
        1.7765107e-03, 5.1651012e-05]], dtype=float32)
 array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
 

In [81]:
# # Testing jh's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'jh' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

44100
[ 0  0  0 ... -2 -2 -4]
441000
[0.0000000e+00 9.1106430e-13 1.9630671e-12 ... 1.0022182e+06 1.0240799e+06
 1.0615945e+06]
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 8.4611114e-05
  1.0807258e-03 3.5964178e-05]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.7385582e-02
  6.6924199e-02 1.9146213e-02]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.0433791e-02
  3.7954453e-02 1.2709111e-01]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.6291668e-06
  3.6419892e-06 2.3978882e-06]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.6837721e-07
  2.3970918e-07 1.3907564e-06]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.9630671e-12
  4.3423734e-07 1.8034008e-07]]


In [35]:
# # Testing larry's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'larry' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

123000
[692 679 666 ... 184 189 194]
1287767
[4.77002444e-13 1.35434209e-12 3.04041102e-12 ... 8.54913188e+05
 9.27614188e+05 1.03139225e+06]
[[1.11041114e-01 4.96798940e-02 1.96016860e+00 ... 1.33353353e-01
  2.24375464e-02 3.58703423e+00]
 [2.22286816e+01 1.78918640e+02 1.47697639e+01 ... 2.13491268e+01
  2.80776024e+00 4.91120377e+01]
 [4.69864655e+01 5.47239418e+01 3.58735237e+01 ... 2.32321334e+00
  9.14043617e+00 5.39548111e+01]
 ...
 [1.16252716e-06 1.05375575e-05 9.63997536e-06 ... 1.32273397e-06
  5.62117566e-06 9.73308374e-07]
 [3.50803657e-06 3.29188254e-07 8.08505035e-07 ... 8.38326315e-08
  4.40014901e-07 8.30593024e-07]
 [1.33116976e-06 1.22837821e-06 9.06501191e-06 ... 8.92284618e-07
  1.61921469e-06 3.30074101e-07]]


In [36]:
# # Testing Chai's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'Chai' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

123000
[  0   0   0 ... 321 320 318]
1416601
[0.0000000e+00 3.0151090e-14 1.1603324e-13 ... 2.8747006e+05 2.9292194e+05
 3.0958022e+05]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.67896110e-01
  2.74241269e-01 5.76840687e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.10411911e+01
  1.21064728e+02 3.28091492e+02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.45097888e-01
  4.54354248e+01 6.00284696e-01]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.28695978e-07
  1.57429679e-06 2.33298379e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.10395683e-06
  8.76923968e-06 5.84162917e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 6.26341262e-07
  1.01666686e-07 4.18995370e-07]]


In [37]:
# Testing yb's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'yb' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

123000
[  0   0   0 ... 136 127 118]
1488002
[0.0000000e+00 6.4757162e-13 3.2806227e-12 ... 4.9208350e+05 4.9369669e+05
 7.2598294e+05]
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.2666391e+01
  6.3406243e+00 3.1554699e+01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.1506744e+02
  8.1753807e+01 6.6353796e+02]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 5.3268345e+01
  8.8878832e+00 3.3352203e+01]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.5038276e-05
  2.3168823e-05 1.2548077e-05]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.1569126e-05
  8.7814112e-07 3.9527908e-06]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 5.8138875e-07
  7.4364159e-10 3.7453237e-08]]
